In [1]:
#importo la libreria di gianluca
import helper


In [2]:
#importo le librerie che mi seriranno
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
#apro sessione con la collection metadata
spark1 = helper.get_session("MPST","metadata")

In [4]:
#leggo il formato mongodb
df = spark1.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [5]:
df.show(5)

+---------+-----+---------------+
|      _id|split|synopsis_source|
+---------+-----+---------------+
|tt0000091|train|      wikipedia|
|tt0000225|train|      wikipedia|
|tt0000230|train|      wikipedia|
|tt0000417|train|           imdb|
|tt0000488|train|      wikipedia|
+---------+-----+---------------+
only showing top 5 rows



In [6]:
#apro sessione con la collection tags
spark = helper.get_session("MPST","tags")

In [7]:
df2 = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [8]:
df2.show(5)

+---------+--------------------+
|      _id|                tags|
+---------+--------------------+
|tt0000091|  paranormal, gothic|
|tt0000225|             fantasy|
|tt0000230|             fantasy|
|tt0000417| psychedelic, satire|
|tt0000488|fantasy, storytel...|
+---------+--------------------+
only showing top 5 rows



In [9]:
#faccio join tra i due dataframe
da =  df.join(df2, on=['_id'])
da.show()

+---------+-----+---------------+--------------------+
|      _id|split|synopsis_source|                tags|
+---------+-----+---------------+--------------------+
|tt0021156|train|      wikipedia|   revenge, romantic|
|tt0032553|train|           imdb|comedy, anti war,...|
|tt0034269|  val|      wikipedia|    romantic, murder|
|tt0038958|train|      wikipedia|              murder|
|tt0039305| test|      wikipedia|plot twist, murde...|
|tt0039431|train|      wikipedia|       psychological|
|tt0039896|train|      wikipedia|   murder, flashback|
|tt0042475| test|      wikipedia|         psychedelic|
|tt0042899|train|      wikipedia|            violence|
|tt0043973| test|      wikipedia|              murder|
|tt0052811| test|      wikipedia|     revenge, murder|
|tt0053320|train|      wikipedia|           melodrama|
|tt0054152|train|           imdb|      comedy, murder|
|tt0054289| test|           imdb|              murder|
|tt0054446|train|      wikipedia|            romantic|
|tt0057261

In [10]:
#apro sessione con la collection plots
spark2 = helper.get_session("MPST","plots")

In [11]:
df3 = spark2.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [12]:
df3.show(5)

+---------+--------------------+--------------------+
|      _id|       plot_synopsis|               title|
+---------+--------------------+--------------------+
|tt0000091|The film opens wi...| Le manoir du diable|
|tt0000225|A widower merchan...| La belle et la bête|
|tt0000230|A prologue in fro...|          Cendrillon|
|tt0000417|At a meeting of t...|Le voyage dans la...|
|tt0000488|Joe is an impover...|The Land Beyond t...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [13]:
#faccio il join tra dataframe plots e quello di prima
data =  da.join(df3, on=['_id'])

In [14]:
data.show(5)

+---------+-----+---------------+---------------+--------------------+--------------------+
|      _id|split|synopsis_source|           tags|       plot_synopsis|               title|
+---------+-----+---------------+---------------+--------------------+--------------------+
|tt0052811| test|      wikipedia|revenge, murder|In 1828 Edinburgh...|The Flesh and the...|
|tt0053320|train|      wikipedia|      melodrama|Alcoholic Bart Hu...|      A Summer Place|
|tt0054152|train|           imdb| comedy, murder|In the first shot...|   Our Man in Havana|
|tt0054289| test|           imdb|         murder|After the death o...|  Il sepolcro dei re|
|tt0054446|train|      wikipedia|       romantic|Kreton (Jerry Lew...|Visit to a Small ...|
+---------+-----+---------------+---------------+--------------------+--------------------+
only showing top 5 rows



In [15]:
data.printSchema()

root
 |-- _id: string (nullable = true)
 |-- split: string (nullable = true)
 |-- synopsis_source: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- plot_synopsis: string (nullable = true)
 |-- title: string (nullable = true)



In [16]:
#sistemo la colonna tags andando a far esplodere gli elementi di quella colonna
import pyspark.sql.functions as f

data = data.select(
        "_id","split","synopsis_source","plot_synopsis","title",
        f.split("tags", ", ").alias("letters"),
        f.posexplode(f.split("tags", ", ")).alias("pos", "val")
    )
data.show(5)

+---------+-----+---------------+--------------------+------------------+--------------------+---+--------+
|      _id|split|synopsis_source|       plot_synopsis|             title|             letters|pos|     val|
+---------+-----+---------------+--------------------+------------------+--------------------+---+--------+
|tt0021156|train|      wikipedia|The film is set i...|           Morocco| [revenge, romantic]|  0| revenge|
|tt0021156|train|      wikipedia|The film is set i...|           Morocco| [revenge, romantic]|  1|romantic|
|tt0032553|train|           imdb|During a battle i...|The Great Dictator|[comedy, anti war...|  0|  comedy|
|tt0032553|train|           imdb|During a battle i...|The Great Dictator|[comedy, anti war...|  1|anti war|
|tt0032553|train|           imdb|During a battle i...|The Great Dictator|[comedy, anti war...|  2|    cult|
+---------+-----+---------------+--------------------+------------------+--------------------+---+--------+
only showing top 5 rows



In [17]:
#elimino le colonne che non mi servono
columns_to_drop = ['letters', 'pos']
data1 = data.drop(*columns_to_drop)
data1.show()

+---------+-----+---------------+--------------------+--------------------+---------------+
|      _id|split|synopsis_source|       plot_synopsis|               title|            val|
+---------+-----+---------------+--------------------+--------------------+---------------+
|tt0061758|train|      wikipedia|The Angels first ...|Hells Angels on W...|           cult|
|tt0061758|train|      wikipedia|The Angels first ...|Hells Angels on W...|         murder|
|tt0061758|train|      wikipedia|The Angels first ...|Hells Angels on W...|      melodrama|
|tt0066735|train|           imdb|Roberto Tobias (M...|4 mosche di vellu...|       neo noir|
|tt0066735|train|           imdb|Roberto Tobias (M...|4 mosche di vellu...|         murder|
|tt0066735|train|           imdb|Roberto Tobias (M...|4 mosche di vellu...|       allegory|
|tt0066735|train|           imdb|Roberto Tobias (M...|4 mosche di vellu...|           cult|
|tt0066735|train|           imdb|Roberto Tobias (M...|4 mosche di vellu...|     

In [18]:
#creo il dataframe per train
train = data1.filter(data1.split == "train")


In [19]:
train.show(5)

+---------+-----+---------------+--------------------+--------------------+----------------+
|      _id|split|synopsis_source|       plot_synopsis|               title|             val|
+---------+-----+---------------+--------------------+--------------------+----------------+
|tt0053320|train|      wikipedia|Alcoholic Bart Hu...|      A Summer Place|       melodrama|
|tt0054152|train|           imdb|In the first shot...|   Our Man in Havana|          comedy|
|tt0054152|train|           imdb|In the first shot...|   Our Man in Havana|          murder|
|tt0054446|train|      wikipedia|Kreton (Jerry Lew...|Visit to a Small ...|        romantic|
|tt0057261|train|      wikipedia|In the midst of a...|   Lord of the Flies|good versus evil|
+---------+-----+---------------+--------------------+--------------------+----------------+
only showing top 5 rows



In [20]:
#vedo quante colonne ha
train.count()

26318

In [21]:
#creo il dataframe per il test
test = data1.filter(data.split == "test")

In [22]:
test.show(5)

+---------+-----+---------------+--------------------+-------------------+-----------+
|      _id|split|synopsis_source|       plot_synopsis|              title|        val|
+---------+-----+---------------+--------------------+-------------------+-----------+
|tt0039305| test|      wikipedia|Leaving a church,...|     Dead Reckoning| plot twist|
|tt0039305| test|      wikipedia|Leaving a church,...|     Dead Reckoning|     murder|
|tt0039305| test|      wikipedia|Leaving a church,...|     Dead Reckoning|  flashback|
|tt0042475| test|      wikipedia|The cat is fishin...|A Fractured Leghorn|psychedelic|
|tt0043973| test|      wikipedia|Insurance investi...|          Roadblock|     murder|
+---------+-----+---------------+--------------------+-------------------+-----------+
only showing top 5 rows



In [23]:
test.count()

7479

In [24]:
#divido la trama in parole e uso le funzioni hastf e idf per transformare le parole in una sequenza di numeri a dimensione fissa
#ogni parola con una diversa importanza
tokenizer = Tokenizer(inputCol="plot_synopsis", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf'
)
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5)
label_stringIdx = StringIndexer(inputCol = "val", outputCol = "target")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])
#pipeline = Pipeline(stages=[tokenizer, hashtf, idf])
pipelineFit = pipeline.fit(train)
train_df = pipelineFit.transform(train)
val_df = pipelineFit.transform(test)
train_df.show(5)


+---------+-----+---------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------+
|      _id|split|synopsis_source|       plot_synopsis|               title|      val|               words|                  tf|            features|target|
+---------+-----+---------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------+
|tt0042899|train|      wikipedia|A car pulls up to...|      Rocky Mountain| violence|[a, car, pulls, u...|(65536,[14,137,53...|(65536,[14,137,53...|   1.0|
|tt0053320|train|      wikipedia|Alcoholic Bart Hu...|      A Summer Place|melodrama|[alcoholic, bart,...|(65536,[120,158,3...|(65536,[120,158,3...|  25.0|
|tt0054152|train|           imdb|In the first shot...|   Our Man in Havana|   comedy|[in, the, first, ...|(65536,[101,223,3...|(65536,[101,223,3...|   7.0|
|tt0054152|train|           imdb|In the first shot...|   Our Man

In [25]:
#train_df.limit(2).toPandas()

In [26]:
#utilizzo la logistic regression per addestrare il modello
#lr = LogisticRegression(maxIter=1, labelCol="target")
#lrModel = lr.fit(train_df)
#predictions = lrModel.transform(val_df)


In [ ]:
#utilizzo la logistic regression per addestrare il modello
lr = LogisticRegression(maxIter=1, labelCol="target")
lrModel = lr.fit(train_df)
predictions = lrModel.transform(val_df)

In [ ]:

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="target")
evaluator.evaluate(predictions)